# Goal of the notebook

The goal of the notebook is to build a dataframe that will get information related to the oil market, holidays and on the stores.

We will then apply an xg boost to this transformed data frame.

In [1]:
import pandas as pd

df_holidays = pd.read_csv('./data/holidays_events.csv')
df_oil = pd.read_csv('./data/oil.csv')
df_stores = pd.read_csv('./data/stores.csv')
df_transactions = pd.read_csv('./data/transactions.csv')
df_train = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")

In [2]:
df_test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [18]:
df_transactions = df_transactions.groupby(['date'], as_index=False)['transactions'].sum().rename(columns = {"transactions" : "tot_transactions"}).merge(df_transactions)
df_transactions['store_weight'] = df_transactions['transactions'] / df_transactions['tot_transactions']
df_transactions.head()

,date,tot_transactions,store_nbr,transactions,store_weight
0,2013-01-01,770,25,770,1.000000
1,2013-01-02,93215,1,2111,0.022647
2,2013-01-02,93215,2,2358,0.025296
3,2013-01-02,93215,3,3487,0.037408
4,2013-01-02,93215,4,1922,0.020619


In [69]:
local_holiday = df_holidays[(df_holidays['transferred'] == False) &
                             (df_holidays['type'] == "Holiday")].merge(df_stores[['store_nbr', 'city']].rename(columns={'city': 'locale_name'}).drop_duplicates())
local_holiday['is_impacted_by_local_holiday'] = True

df_transactions_local= local_holiday[['date', 'store_nbr','is_impacted_by_local_holiday']].merge(df_transactions, on = ['store_nbr', 'date'], how = "right")
df_transactions_local['is_impacted_by_local_holiday'] = df_transactions_local['is_impacted_by_local_holiday'].fillna(False)
local_holiday_bis = df_holidays[(df_holidays['transferred'] == False) &
                             (df_holidays['type'] == "Transfer")]\
                              .merge(df_stores[['store_nbr', 'city']].rename(columns={'city': 'locale_name'}).drop_duplicates())
local_holiday_bis['is_impacted_by_local_holiday'] = True

df_transactions_local_bis= local_holiday[['date', 'store_nbr','is_impacted_by_local_holiday']]\
   .merge(df_transactions, on = ['store_nbr', 'date'], how = "right")
df_transactions_local_bis['is_impacted_by_local_holiday'] = df_transactions_local_bis['is_impacted_by_local_holiday'].fillna(False)
df_transactions_local_bis = df_transactions_local_bis[df_transactions_local_bis['is_impacted_by_local_holiday']]

df_transactions_local = pd.concat([df_transactions_local, df_transactions_local_bis])


regional_holiday = df_holidays[(df_holidays['transferred'] == False) &
                             (df_holidays['type'] == "Holiday")].merge(df_stores[['store_nbr', 'state']].rename(columns={'state': 'locale_name'}).drop_duplicates())
regional_holiday['is_impacted_by_regional_holiday'] = True

df_transactions_regional= regional_holiday[['date', 'store_nbr','is_impacted_by_regional_holiday']].merge(df_transactions, on = ['store_nbr', 'date'], how = "right")
df_transactions_regional['is_impacted_by_regional_holiday'] = df_transactions_regional['is_impacted_by_regional_holiday'].fillna(False)

date_min, date_max = (df_oil[df_oil['dcoilwtico'] > 60].sort_values(by = "date")['date'].min(),
       df_oil[df_oil['dcoilwtico'] > 60].sort_values(by = "date")['date'].max())

df_transactions_local['oil_impact'] = 0

df_transactions_local.loc[(df_transactions_local['date'] > date_max), "oil_impact"] = 1

In [72]:
df_transactions_ = df_transactions_local.merge(df_transactions_regional[['date', 'store_nbr','is_impacted_by_regional_holiday']].drop_duplicates()).drop_duplicates()

In [73]:
df_transactions_[df_transactions_['is_impacted_by_regional_holiday']].head()

,date,store_nbr,is_impacted_by_local_holiday,tot_transactions,transactions,store_weight,oil_impact,is_impacted_by_regional_holiday
4106,2013-04-01,12,False,81678,1313,0.016075,0,True
4107,2013-04-01,13,False,81678,1125,0.013774,0,True
8068,2013-06-25,15,False,70549,1469,0.020822,0,True
10007,2013-08-05,43,True,82080,1411,0.017191,0,True
14351,2013-11-06,5,False,76148,1400,0.018385,0,True


In [98]:
df_holidays__ = df_holidays[(df_holidays["locale"] == "National") & (df_holidays["transferred"] == False) & 
                            (df_holidays["type"].isin(['Holiday', 'Additional', 'Bridge', 'Work Day', 'Event']))].copy()

# Add a column with True to indicate the presence of each type
df_holidays__['is_present'] = True

# Pivot the DataFrame, with date as the index and each type as a column
df_pivot = df_holidays__.pivot_table(index='date', columns='type', values='is_present', fill_value=False)
df_pivot.columns = ['national_' + col for col in df_pivot.columns]
# Display the result
df_pivot.reset_index(inplace=True)
for cols in df_pivot.columns:
   df_pivot[cols] = df_pivot[cols].replace({1.0: True})

**Not transferred days**

In [97]:
df_holidays__ = df_holidays[(df_holidays["locale"] == "National") & (df_holidays["transferred"] == True)].copy()

# Add a column with True to indicate the presence of each type
df_holidays__['is_present'] = True
df_holidays__['type'] = "Holiday"

# Pivot the DataFrame, with date as the index and each type as a column
df_pivot_bis = df_holidays__.pivot_table(index='date', columns='type', values='is_present', fill_value=False)
df_pivot_bis.columns = ['national_' + col for col in df_pivot_bis.columns]
# Display the result
df_pivot_bis.reset_index(inplace=True)
for cols in df_pivot_bis.columns:
   df_pivot_bis[cols] = df_pivot_bis[cols].replace({1.0: True})

df_pivot_bis['national_Additional'] = False
df_pivot_bis['national_Bridge'] = False
df_pivot_bis['national_Event'] = False
df_pivot_bis['national_Work Day'] = False

df_pivot = pd.concat([df_pivot, df_pivot_bis])
df_pivot.head()

,date,national_Additional,national_Bridge,national_Event,national_Holiday,national_Work Day
0,2012-08-10,False,False,False,True,False
1,2012-11-02,False,False,False,True,False
2,2012-11-03,False,False,False,True,False
3,2012-12-21,True,False,False,False,False
4,2012-12-22,True,False,False,False,False


In [102]:
df_pivot.columns

Index(['date', 'national_Additional', 'national_Bridge', 'national_Event',
       'national_Holiday', 'national_Work Day'],
      dtype='object')

In [99]:
df_transactions_ = df_transactions_.merge(df_pivot, how = 'left')

In [103]:
for cols in ['national_Additional', 'national_Bridge', 'national_Event',
       'national_Holiday', 'national_Work Day']:
   
   df_transactions_[cols] = df_transactions_[cols].fillna(False)

In [107]:
df_transactions_.drop(index=df_transactions_.index[0], axis=0, inplace=True)
df_transactions_[df_transactions_.store_weight == df_transactions_.store_weight.max()]

,date,store_nbr,is_impacted_by_local_holiday,tot_transactions,transactions,store_weight,oil_impact,is_impacted_by_regional_holiday,national_Additional,national_Bridge,national_Event,national_Holiday,national_Work Day
34046,2015-01-01,25,False,2202,2202,1.0,0,False,False,False,False,True,False
71391,2017-01-01,25,False,1642,1642,1.0,1,False,False,False,False,False,False


Remove 1st row

In [112]:
df_train[df_train['onpromotion'] == 425]

,id,date,store_nbr,family,sales,onpromotion
2964402,2964402,2017-07-26,35,GROCERY I,2123.0,425


In [116]:
df_train.drop('id', axis=1, inplace=True)
len(df_train)

3000888

In [118]:
df_train.groupby(['date', "store_nbr"], as_index=False).agg({"sales" : "sum",
                                             "onpromotion" : "sum"}).rename(columns={"sales" : "tot_sales_per_store",
                                                                                     "onpromotion": "tot_promotion_per_store"})\
         .merge(df_train)

,date,store_nbr,tot_sales_per_store,tot_promotion_per_store,family,sales,onpromotion
0,2013-01-01,1,0.000,0,AUTOMOTIVE,0.000,0
1,2013-01-01,1,0.000,0,BABY CARE,0.000,0
2,2013-01-01,1,0.000,0,BEAUTY,0.000,0
3,2013-01-01,1,0.000,0,BEVERAGES,0.000,0
4,2013-01-01,1,0.000,0,BOOKS,0.000,0
...,...,...,...,...,...,...,...
3000883,2017-08-15,54,12666.858,204,POULTRY,59.619,0
3000884,2017-08-15,54,12666.858,204,PREPARED FOODS,94.000,0
3000885,2017-08-15,54,12666.858,204,PRODUCE,915.371,76
3000886,2017-08-15,54,12666.858,204,SCHOOL AND OFFICE SUPPLIES,0.000,0


In [120]:
df_train.pivot(index=['date', "store_nbr"], columns="family", values="sales").reset_index()

family,date,store_nbr,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,...,MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
0,2013-01-01,1,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000
1,2013-01-01,2,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000
2,2013-01-01,3,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000
3,2013-01-01,4,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000
4,2013-01-01,5,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90931,2017-08-15,50,11.0,0.0,16.0,3661.0,0.0,466.130,45.0,1322.0,...,13.0,425.802,396.0,9.0,22.0,356.510,73.852,2027.374,219.0,15.314
90932,2017-08-15,51,11.0,0.0,2.0,3860.0,0.0,704.801,16.0,1357.0,...,13.0,567.849,234.0,15.0,25.0,686.941,91.976,5031.190,0.0,52.876
90933,2017-08-15,52,2.0,0.0,4.0,3744.0,0.0,756.481,15.0,1412.0,...,19.0,435.828,278.0,9.0,20.0,397.340,104.418,3803.368,17.0,7.000
90934,2017-08-15,53,9.0,0.0,7.0,1646.0,0.0,380.199,4.0,706.0,...,5.0,185.798,153.0,5.0,5.0,148.015,37.820,1432.489,0.0,5.000


# Final dataframe

In [110]:
df_train[["date", "store_nbr", 'family', "sales", "onpromotion"]].drop_duplicates().merge(df_transactions_, how="right").drop_duplicates()

,date,store_nbr,family,sales,onpromotion,is_impacted_by_local_holiday,tot_transactions,transactions,store_weight,oil_impact,is_impacted_by_regional_holiday,national_Additional,national_Bridge,national_Event,national_Holiday,national_Work Day
0,2013-01-02,3,AUTOMOTIVE,10.000,0,False,93215,3487,0.037408,0,False,False,False,False,False,False
1,2013-01-02,3,BABY CARE,0.000,0,False,93215,3487,0.037408,0,False,False,False,False,False,False
2,2013-01-02,3,BEAUTY,15.000,0,False,93215,3487,0.037408,0,False,False,False,False,False,False
3,2013-01-02,3,BEVERAGES,4486.000,0,False,93215,3487,0.037408,0,False,False,False,False,False,False
4,2013-01-02,3,BOOKS,0.000,0,False,93215,3487,0.037408,0,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2755000,2017-08-15,54,POULTRY,59.619,0,False,86561,802,0.009265,1,False,False,False,False,False,False
2755001,2017-08-15,54,PREPARED FOODS,94.000,0,False,86561,802,0.009265,1,False,False,False,False,False,False
2755002,2017-08-15,54,PRODUCE,915.371,76,False,86561,802,0.009265,1,False,False,False,False,False,False
2755003,2017-08-15,54,SCHOOL AND OFFICE SUPPLIES,0.000,0,False,86561,802,0.009265,1,False,False,False,False,False,False
